In [ ]:
#실거래가랑 병합 코드

In [7]:
import pandas as pd
import requests
from tqdm import tqdm

# 카카오맵 API 키
API_KEY = "f3da4534e20ed82ac1571f3e8eba33c0"

# 데이터 파일 경로
real_estate_file_path = "거리계산(중복제거)/실거래가 연도별 병합/아파트_전월세_종로구.csv"
distance_calculation_file_path = "거리계산(중복제거)/거리계산_중간_병합/편의점_공공도서관_지하철_버스_아파트.xlsx"

# 데이터 불러오기
real_estate_data = pd.read_csv(real_estate_file_path, encoding= 'euc-kr')
distance_data = pd.read_excel(distance_calculation_file_path)

# 결과 데이터프레임
merged_data = pd.DataFrame(columns=real_estate_data.columns)

# 각 실거래가 데이터에 대해 도로명을 위도와 경도로 바꾸고 거리계산 파일에 해당하는 위도 경도와 일치하는 경우 데이터 병합
for index, row in tqdm(real_estate_data.iterrows(), total=real_estate_data.shape[0]):
    address = row['도로명']
    if pd.notna(address):  # 도로명이 NaN이 아닌 경우에만 처리
        try:
            address = str(address).strip()  # 도로명주소를 문자열로 변환 후 공백 제거
            if address:  # 도로명이 비어있지 않은 경우에만 처리
                # 카카오맵 API를 이용하여 주소로부터 위도와 경도 정보 가져오기
                response = requests.get(f"https://dapi.kakao.com/v2/local/search/address.json?query={address}", headers={"Authorization": f"KakaoAK {API_KEY}"})
                data = response.json()
                if data['documents']:
                    latitude = round(float(data['documents'][0]['y']), 6)
                    longitude = round(float(data['documents'][0]['x']), 6)
                    # 거리계산 파일에서 해당 주소와 일치하는 행을 찾아서 병합
                    matching_rows = distance_data[(round(distance_data['단지 위도'], 6) == latitude) & (round(distance_data['단지 경도'], 6) == longitude)]
                   
                    if not matching_rows.empty:
                        # 거리계산 파일의 데이터를 실거래가 데이터의 해당 행에 추가
                        for column in matching_rows.columns:
                            real_estate_data.at[index, column] = matching_rows.iloc[0][column]

        except Exception as e:
            print(f"Error occurred while processing row {index}: {e}")
    
    # 병합된 데이터를 결과 데이터프레임에 추가
    merged_data = pd.concat([merged_data, real_estate_data.loc[[index]]], ignore_index=True)
merged_data.to_excel("아파트_전월세_종로구_병합.xlsx", index=False)

# 결과 출력
#print(merged_data)


100%|██████████████████████████████████████████████████████████████████████████████| 9670/9670 [14:10<00:00, 11.38it/s]


In [ ]:
#실거래가 파일 좌표변환 코드

In [3]:
import pandas as pd
import requests
from tqdm import tqdm

def address_to_coords(api_key, address):
    """
    Convert address to coordinates using Kakao Maps API.

    Parameters:
        api_key (str): Your Kakao Maps API key.
        address (str): Address to convert to coordinates.

    Returns:
        tuple: A tuple containing latitude and longitude.
    """
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    
    if "documents" in data and data["documents"]:
        latitude = data["documents"][0]["y"]
        longitude = data["documents"][0]["x"]
        return latitude, longitude
    else:
        return None, None

def main():
    api_key = "b3d07b3bc9d36308dfc4b023582e291d"  # Replace with your own Kakao Maps API key

    # Load data and remove duplicate rows
    apartmon_23 = pd.read_csv('거리계산(중복제거)/DB_변수추가/연립다세대_매매.csv', encoding='euc-kr')[98060:98090]
    apartmon_23 = apartmon_23[['단지명', '번지', '도로명']]
    apartmon_23 = apartmon_23.drop_duplicates()

    # Create empty lists to store data
    complex_names = []
    bungji = []  # New list for '번지'
    road_names = []
    latitudes = []
    longitudes = []
    
    coords_df = pd.DataFrame(columns=['단지명', '번지', '도로명', '단지 위도', '단지 경도'])

    # Iterate over each row in the dataframe
    for index, row in tqdm(apartmon_23.iterrows(), total=apartmon_23.shape[0]):
        address = row['도로명']  # Use only '도로명' for address conversion
        complex_name = row['단지명']
        bungji_val = row['번지']  # Get value from '번지' column
        latitude, longitude = address_to_coords(api_key, address)
        
        # Append data to lists
        complex_names.append(complex_name)
        bungji.append(bungji_val)  # Append '번지' value
        road_names.append(row['도로명'])
        latitudes.append(latitude)
        longitudes.append(longitude)
        
        # Update coords_df outside the loop
        coords_df = pd.DataFrame({
            '단지명': complex_names,
            '번지': bungji,  # Use '번지' list
            '도로명': road_names,
            'latitude': latitudes,
            'longitude': longitudes
        })
        #coords_df.to_csv("거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표수정7.csv", index=False)
    print("모든 반복이 완료되었습니다.")
    print(coords_df)

# Run the main function
if __name__ == "__main__":
    main()


100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  7.82it/s]

모든 반복이 완료되었습니다.
          단지명      번지          도로명          latitude         longitude
0  리츠빌(639-3)   639-3  논현로97길 19-4  37.5026491687873  127.034000125207
1        삼경빌라  674-25  언주로93길 14-5  37.5041074604921  127.040524718334


In [ ]:
#단지위도, 경도 빈셀 제거 

In [47]:
import pandas as pd

# CSV 파일을 읽어옵니다.
df1 = pd.read_csv("거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표추가.csv", encoding='euc-kr')

# latitude 열의 빈 셀(NaN)을 포함하는 행을 삭제합니다.
df1 = df1.dropna(subset=['단지 위도'])

# 수정된 DataFrame을 확인합니다.
print(df1)

# 필요하다면 수정된 DataFrame을 다시 CSV 파일로 저장합니다.
df1.to_csv("거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표추가수정.csv", index=False)


                 건물명      번지            도로명      단지 위도       단지 경도
0             청운빌라A동  108-13        자하문로 98  37.584745  126.970481
1           풍림팍사이드빌라   134-2      자하문로 99-3  37.584812  126.969867
2            청운벽산빌리지       1  자하문로36길 16-14  37.591583  126.968017
3           (52-128)  52-128    자하문로33다길 14  37.587021  126.967957
4              신구파인힐   04-07     자하문로36길 22  37.590300  126.967834
...              ...     ...            ...        ...         ...
61250        자하문한숲빌라  208-29    자하문로40가길 13  37.594877  126.964430
61251           조형빌라  179-15   자하문로42길 29-5  37.599259  126.961658
61252  (Atomosphere)     275     창의문로10길 36  37.593040  126.964536
61253    시아부암아이윌(A동)  182-27       자하문로 288  37.599051  126.960919
61254           삼산빌라   34-15      홍지문2가길 16  37.600243  126.954852

[59486 rows x 5 columns]


In [ ]:
#실거래가 병합 - 연도 구분 x 

In [ ]:
import pandas as pd

# 5개의 CSV 파일 경로 지정
file_paths = [
    '실거래가(변수정리).csv/연립다세대(매매)_실거래가_2019.csv',
    '실거래가(변수정리).csv/연립다세대(매매)_실거래가_2020.csv',
    '실거래가(변수정리).csv/연립다세대(매매)_실거래가_2021.csv',
    '실거래가(변수정리).csv/연립다세대(매매)_실거래가_2022.csv',
    '실거래가(변수정리).csv/연립다세대(매매)_실거래가_2023.csv'
]

# 빈 리스트를 생성하여 각 CSV 파일을 데이터프레임으로 불러와 저장
dataframes = []

for file_path in file_paths:
    df = pd.read_csv(file_path, encoding = 'euc-kr')
    dataframes.append(df)

# 모든 데이터프레임을 하나로 병합
merged_df = pd.concat(dataframes, ignore_index=True)

# 결과 확인 (필요에 따라 출력)
print(merged_df)

# 병합된 데이터프레임을 새로운 CSV 파일로 저장 (필요에 따라)
output_file = '실거래가(변수정리).csv/연립다세대_매매_전체.csv'  # 저장할 파일 경로와 이름
merged_df.to_csv(output_file, index=False)


                  시군구      번지        건물명  전용면적(㎡)  대지권면적(㎡)    계약년월  계약일  \
0       서울특별시 종로구 청운동  108-13     청운빌라A동   112.50   89.8500  201909   23   
1       서울특별시 종로구 청운동   134-2   풍림팍사이드빌라    84.43   59.4495  201909   24   
2       서울특별시 종로구 청운동       1    청운벽산빌리지   141.39  137.0300  201909   25   
3       서울특별시 종로구 청운동   134-2   풍림팍사이드빌라    53.83   37.9020  201909   30   
4       서울특별시 종로구 청운동       1    청운벽산빌리지   176.40  170.9500  201908   15   
...               ...     ...        ...      ...       ...     ...  ...   
219608  서울특별시 종로구 신영동   03-03       신영빌라   104.05   87.4667  202301    6   
219609  서울특별시 종로구 신영동   212-1       완성빌라    42.84   69.7000  202308   27   
219610  서울특별시 종로구 신영동     209  북악더테라스6단지    84.95   90.6300  202302   24   
219611  서울특별시 종로구 신영동   103-1      현대빌리지    86.29   57.3700  202307    3   
219612  서울특별시 종로구 신영동   150-1       자하주택   109.09  128.9000  202306   30   

       거래금액(만원)  층  건축년도            도로명  
0        67,000  3  2000        자하문로 98  
1  

In [ ]:
#실거래가 파일 + 위경도 추가

In [4]:
import pandas as pd

# 첫 번째 CSV 파일 불러오기
df1 = pd.read_csv("거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_전체.csv") #encoding = 'euc-kr'

# 두 번째 CSV 파일 불러오기
df2 = pd.read_csv("거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표추가.csv",encoding = 'euc-kr')

# 도로명이 있는 경우와 없는 경우로 데이터를 분리
df1_with_road = df1.dropna(subset=['도로명'])
df1_without_road = df1[df1['도로명'].isnull()]

# 도로명이 있는 경우와 없는 경우를 각각 병합
merged_df_with_road = pd.merge(df1_with_road,df2, on=['건물명', '도로명', '번지'], how='left')
merged_df_without_road = pd.merge(df1_without_road,df2, on=['건물명', '번지'], how='left')
# 두 결과를 합치기
merged_df = pd.concat([merged_df_with_road, merged_df_without_road])
merged_df.drop(columns=['도로명_x', '도로명_y'], inplace=True)
merged_df.dropna(subset=['단지 위도', '단지 경도'], inplace=True)
merged_df
# 결과 출력
merged_df.to_csv('거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표생성.csv', index=False)


In [ ]:
#실거래가 파일 + 거리계산 병합

In [52]:
import pandas as pd
from tqdm import tqdm

counted_pharmacy_path = "거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_좌표생성.csv"
counted_library_path = "거리계산(중복제거)/거리계산_중간_병합/전체거리계산_연립다세대.xlsx"

# 데이터 불러오기
counted_pharmacy_data = pd.read_csv(counted_pharmacy_path, encoding = 'euc-kr')
counted_library_data = pd.read_excel(counted_library_path)

counted_pharmacy_data['단지 위도'] = counted_pharmacy_data['단지 위도'].round(6)
counted_pharmacy_data['단지 경도'] = counted_pharmacy_data['단지 경도'].round(6)

counted_library_data['단지 위도'] = counted_library_data['단지 위도'].round(6)
counted_library_data['단지 경도'] = counted_library_data['단지 경도'].round(6)

merged_data = pd.merge(counted_pharmacy_data, counted_library_data, on=['단지명','단지 위도', '단지 경도'], how='inner')

# 결과 확인
print(merged_data)

# 결과를 엑셀 파일로 
merged_data.to_csv('거리계산(중복제거)/실거래가 연도별 병합/연립다세대_매매_최종병합.csv', index=False)


                  시군구      번지            단지명  전용면적(㎡)  대지권면적(㎡)    계약년월  계약일  \
0       서울특별시 종로구 청운동  108-13         청운빌라A동   112.50   89.8500  201909   23   
1       서울특별시 종로구 청운동  108-13         청운빌라A동   122.16   97.5700  202303   17   
2       서울특별시 종로구 청운동   134-2       풍림팍사이드빌라    84.43   59.4495  201909   24   
3       서울특별시 종로구 청운동   134-2       풍림팍사이드빌라    53.83   37.9020  201909   30   
4       서울특별시 종로구 청운동   134-2       풍림팍사이드빌라    53.83   37.9020  202001   20   
...               ...     ...            ...      ...       ...     ...  ...   
199096  서울특별시 종로구 부암동  208-29        자하문한숲빌라    82.92   45.2400  202306   30   
199097  서울특별시 종로구 부암동  179-15           조형빌라   200.15  236.8400  202305   23   
199098  서울특별시 종로구 부암동     275  (Atomosphere)   123.38  100.2000  202306    8   
199099  서울특별시 종로구 부암동  182-27    시아부암아이윌(A동)    73.08   41.2500  202309   18   
199100  서울특별시 종로구 홍지동   34-15           삼산빌라    84.60   82.8750  202310    7   

       거래금액(만원)  층  건축년도  ... 중학교 거리(m)